# Veeper: Create, Run, and Collate

This is a generalized code that can bring in any spectra, save it to its own components folder, and save the corresponding fits

Install: https://github.com/mattcwilde/veeper

This notebook is supposed to be run after asborption features have been identified using pyigm_igm guesses

Install: https://github.com/pyigm/pyigm

Here are some notes for the future

#### Running this code in Step 3 will produce a FitInspection.pdf file with a preview of your voigt fits: 
- Red line = total 
- dashed line = particular component 
- the order is the redshift 

#### If the file needs to be changed (editing.ipynb): 

In [1]:
##--- Step one 
from joebvp.utils import pyigm_to_veeper
from joebvp.VPmeasure import batch_fit
from joebvp import VPmeasure
import glob
import os
import astropy.units as u

read_sets: Using set file -- 
  /Users/samgarza/linetools/linetools/lists/sets/llist_v1.3.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
joebvp.VPmeasure: No local joebvp_cfg.py found, using default cfg.py file from joebvp.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/samgarza/linetools/linetools/data/lines/verner96_tab1.fits.gz
joebvp.makevoigt: No local joebvp_cfg.py found, usin

#### Paths for each of my spectra: 
- ##### NGC 3489 / J1101: 
    - json_path =  '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1101+1429/SamIDs_jess_Nov29_2022.json' 
    - norm = 'J1101+1429_nbin3_jwnorm.fits'

- ##### NGC 4026 / J1159: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Dec1_2022.json'
    - norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
    
- ##### NGC 4736 / J1249: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1249+4122/SamIDs_Nov28_2022.json'
    - norm = 'J1249+4122_nbin3_visit12_jwnorm.fits'
    
- ##### NGC 3414 / 1051: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1051+2805/SamIDs_Nov29_2022.json'
    - norm = 'J1051+2805_nbin3_jwnorm.fits'
    
- ##### NGC 3627 / J1123: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1123+1257/SamIDs_Nov29_2022.json'
    - norm = 'J1123+1257_nbin3_BASIC_jwnorm.fits'

- ##### NGC 1097 / J0246: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J0246-3007/SamIDs_Nov28_2022.json'
    - norm = 'J0246-3007_nbin3_allvisits_jwnorm.fits'

- ##### NGC 4564 / J1235: 
    - json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1235+1123/SamIDs_Nov28_2022.json'
    - norm = 'J1235+1123_twovisits_m3_jwnorm.fits'

In [2]:
#Step 2: Imput the specifications 
    # basically takes the gueses and the norm to start the process
    #dv_expand is how much you can expand the redlines to be of some problem 

#These will change for each galaxy/line of sight  
#path to specific .json file
json_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/SamIDs_meth3_Dec1_2022.json'

# normalization file that you want to use
norm = 'J1159+5106_nbin3_jwnorm_meth3.fits'
norm_path = '/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159+5106/J1159+5106_nbin3_jwnorm_meth3.fits' #path to normalization file 

pyigm_to_veeper(json_path, norm, dv_expand=50*u.km/u.s)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [3]:
input_files = glob.glob('component_groups/*.txt')

#This was 10 in the other one but I want to make this 20 since it skipped something that was 15 before 
max_len = 20 #skip over files where there are more than this many lines to fit 
redo = False

to_do = []
for input_file in input_files:
    if 'rchi2' in input_file:
        continue
    if not redo:
        #already done?
        if os.path.exists(input_file.replace('.txt', '.VP')): 
            print('Skipping: Already done {}'.format(input_file))
            continue
    
    #too long?
    with open(input_file, 'r') as f:
        length = len(f.readlines())
        if length > max_len:
            print('Skipping: Too long ({} rows) {}'.format(length, input_file))
            continue
            
    to_do.append(input_file)

In [4]:
###--- Step 3: Run the fit
VPmeasure.batch_fit(norm_path, filelist=to_do, filepath='./component_groups/')

`ftol` termination condition is satisfied.
Function evaluations 16, initial cost 7.8064e+01, final cost 6.1119e+01, first-order optimality 4.58e-04.

Fit results: 

1215.67	 0.308080	 12.843	 15.529	 4.1171
 	  	  	 0.213	 13.388	 7.027 

1215.67	 0.308510	 13.501	 53.026	 -6.1678
 	  	  	 0.09	 14.254	 7.749 

1215.67	 0.309450	 13.705	 89.700	 -45.946
 	  	  	 0.059	 15.837	 10.117 


Reduced chi-squared: 1.527984
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_9.txt
Line parameters written to:
component_groups/input_group_9.VP
Wrote spectrum to component_groups/input_group_9_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
joebvp.joebvpfit: No local joebvp_cfg.py found, using default cfg.py file form joebvp.


Wrote spectrum to component_groups/input_group_9_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 19, initial cost 7.4725e+01, final cost 6.3414e+01, first-order optimality 7.80e-04.

Fit results: 

1215.67	 0.303990	 13.611	 69.500	 2.4861
 	  	  	 0.861	 38.388	 67.462 

1215.67	 0.304650	 13.657	 85.600	 -41.317
 	  	  	 0.828	 103.24	 110.265 

1215.67	 0.305340	 12.937	 58.800	 -19.898
 	  	  	 0.442	 58.643	 45.101 


Reduced chi-squared: 1.647116
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_8.txt
Line parameters written to:
component_groups/input_group_8.VP
Wrote spectrum to component_groups/input_group_8_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_8_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 7, initial cost 8.1463e+01, final cost 5.6428e+01, first-order optimality 2.19e-04.

Fit results: 

1393.76	 -0.000140	 13.471	 45.500	 -18.759
 	  	  	 0.035	 4.934	 3.436 

1402.77	 -0.000140	 13.471	 45.500	 -18.759
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 1.567458
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_0.txt
Line parameters written to:
component_groups/input_group_0.VP
Wrote spectrum to component_groups/input_group_0_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_0_inspect.fits
`ftol` termination condition is satisfied.
Function evaluations 9, initial cost 7.1441e+01, final cost 6.9712e+01, first-order optimality 1.65e-04.

Fit results: 

1393.76	 0.002910	 12.731	 20.000	 -3.1313
 	  	  	 0.124	 10.033	 6.007 

1402.77	 0.002910	 12.731	 20.000	 -3.1313
 	  	  	 0.0	 0.0	 0.0 

1393.76	 0.003340	 13.051	 53.300	 -4.4396
 	  	  	 0.089	 14.7	 9.63 

1402.77	 0.003340	 13.051	 53.300	 -4.4396
 	  	  	 0.0	 0.0	 0.0 


Reduced chi-squared: 1.315318
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_1.txt
Line parameters written to:
component_groups/input_group_1.VP
Wrote spectrum to component_groups/input_group_1_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_1_inspect.fits


...

Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_10_inspect.fits
There are no valid pixels to fit in the region. Moving on.
`ftol` termination condition is satisfied.
Function evaluations 19, initial cost 2.1192e+03, final cost 1.5836e+02, first-order optimality 1.86e-01.

Fit results: 

1670.79	 -0.000180	 14.137	 36.800	 -23.918
 	  	  	 0.209	 4.255	 1.699 


Reduced chi-squared: 6.598126
Iteration 1 -
Fit converged after 1 outer iterations.
VPmeasure: Fit converged: component_groups/input_group_13.txt
Line parameters written to:
component_groups/input_group_13.VP
Wrote spectrum to component_groups/input_group_13_VPmodel.fits
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


Wrote spectrum to component_groups/input_group_13_inspect.fits
There are no valid pixels to fit in the region. Moving on.

VPmeasure: 13/15 fits converged, 0/15 failed (see log for details).

VPmeasure: concatenating individual outputs and creating figures for inspection.
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/Users/samgarza/linetools/linetools/spectra/lsf.py:551: UserWarning: LSF may result from extrapolation outside wavelength range characterized for current grating.
  "LSF may result from extrapolation outside wavelength range characterized for current grating.")
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(super(), op)(other)
/Users/samgarza/opt/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:1157: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr

Wrote spectrum to FitInspection.fits
VPmeasure: Done.


In [5]:
#make the directory to hold creates files for each qso/LOS 

parent_path = '/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/' #where I want the created files to be put in general 
qso_name = 'J1159/' #need to change this to match each QSO LOS name; where I want the created files to be put in specifically 

#set directory name
directory = qso_name
# Path
path = os.path.join(parent_path, directory)
#Make it happen
os.mkdir(path)

In [6]:
#Step 4: Collate
import pandas as pd

vp_file_list = glob.glob('component_groups/*.VP')
vp_file_list

['component_groups/input_group_2.VP',
 'component_groups/input_group_6.VP',
 'component_groups/input_group_7.VP',
 'component_groups/input_group_3.VP',
 'component_groups/input_group_8.VP',
 'component_groups/input_group_13.VP',
 'component_groups/input_group_9.VP',
 'component_groups/input_group_10.VP',
 'component_groups/input_group_14.VP',
 'component_groups/input_group_4.VP',
 'component_groups/input_group_0.VP',
 'component_groups/input_group_1.VP',
 'component_groups/input_group_5.VP']

In [7]:
vp_dfs = []

for vp_file in vp_file_list:
    vp_df = pd.read_csv(vp_file, sep='|')
    vp_dfs.append(vp_df)
    
all_vp_df = pd.concat(vp_dfs, ignore_index=True)
all_vp_df

,specfile,restwave,zsys,col,sigcol,bval,sigbval,vel,sigvel,nflag,...,vlim1,vlim2,wobs1,wobs2,pix1,pix2,z_comp,trans,rely,comment
0,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1190.4158,0.19946,13.340,2.600000e-02,47.000,3.835000e+00,-7.168,2.537,2,...,-68.4444,68.4444,1427.584333,1428.127891,1215,1230,0.19943,SiII,b,--
1,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1193.2897,0.19946,13.340,0.000000e+00,47.000,0.000000e+00,-7.168,0.000,2,...,-68.4444,68.4444,1431.030805,1431.575675,1309,1324,0.19943,SiII,b,--
2,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1260.4221,0.19946,13.340,0.000000e+00,47.000,0.000000e+00,-7.168,0.000,2,...,-68.4444,68.4444,1511.538105,1512.113629,3503,3519,0.19943,SiII,b,--
3,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1304.3702,0.19946,13.340,0.000000e+00,47.000,0.000000e+00,-7.168,0.000,2,...,-68.4444,68.4444,1564.242059,1564.837649,4940,4956,0.19943,SiII,b,--
4,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1548.2040,0.00295,13.101,1.510000e-01,8.623,9.335000e+00,11.009,4.415,2,...,-53.8663,53.8663,1552.493239,1553.049598,4620,4635,0.00299,C IV,b,--
5,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1550.7810,0.00295,13.101,0.000000e+00,8.623,0.000000e+00,11.009,0.000,2,...,-53.8663,53.8663,1555.077379,1555.634663,4690,4705,0.00299,C IV,b,--
6,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1548.2040,0.00340,13.513,7.900000e-02,35.200,9.396000e+00,-32.763,6.209,3,...,-77.2864,77.2864,1553.063952,1553.862205,4635,4657,0.00329,C IV,b,--
7,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1550.7810,0.00340,13.513,0.000000e+00,35.200,0.000000e+00,-32.763,0.000,3,...,-77.2864,77.2864,1555.649042,1556.448623,4706,4727,0.00329,C IV,b,--
8,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1215.6700,0.27956,13.339,1.520000e-01,28.156,1.073300e+01,0.280,7.526,4,...,-43.3272,43.3272,1555.342769,1555.694156,4697,4707,0.27956,H I,b,--
9,/Users/samgarza/Dropbox/COS-HolesHSTdata/J1159...,1215.6700,0.28000,13.892,8.600000e-02,60.879,1.718500e+01,4.056,7.034,5,...,-52.6953,52.6953,1555.839639,1556.267002,4711,4722,0.28001,H I,b,--


In [8]:
#Save collate informtion to csv file 
all_vp_df.to_csv(path + 'all_vp.csv')

In [9]:
import shutil

#Move over the component_groups
shutil.move(parent_path + 'component_groups/', path)

'/Users/samgarza/Dropbox/Mac/Documents/COS-Holes/veeper_and_voigt/J1159/component_groups'

In [10]:
#list of created files that need to be moved over to QSO LOS specific directory 
files_to_move = ['_VP_log.dat', 'all_VP.txt', 'compiledVPoutputs.dat', 'FitInspection.fits', 'FitInspection.pdf']

# iterate files
for file in files_to_move:
    # construct full file path
    source = parent_path + file
    destination = path + file
    # move file
    shutil.move(source, destination)
    print('Moved:', file)

Moved: _VP_log.dat
Moved: all_VP.txt
Moved: compiledVPoutputs.dat
Moved: FitInspection.fits
Moved: FitInspection.pdf
